In [22]:
import datetime

class Account:
    def __init__(self, account_holder, balance=0.0):
        if balance < 0:
            raise ValueError("Начальный баланс не может быть отрицательным")
        # здесь я решила использовать raise, а не print, так как raise служит для того, чтобы создать ошибку, программа останавливается (если ошибку не перехватить), а print просто выводит текст. По условию задачи минусовой баланс не возможен, поэтому более коррктно будет использование raise.
        # теперь создадим атрибут объекта нашего класса 
        self.holder = account_holder
        # в атрибут self.holder мы будем сохранять значение параметра объекта account_holder, который был передан при создании объекта. Это нужно, чтобы помнить, кому принадлежит счет.
        # создаем приватный атрибут _balance. Чтобы сделать его приватным, мы добавили нижнее подчеркивание в начале. В него мы будем сохранять начальный баланс.
        self.__balance = float(balance)
        # теперь создадим атрибут operations_history - хранилище для истории операций. Сначала он будет пустым, но при каждом deposit или withdraw мы будем добавлять в этот список словарь с информацией об операции
        self.operations_history = []
    
    # добавляем метод deposit
    def deposit(self, amount):
        if amount <= 0:
            raise ValueError("Сумма пополнения должна быть положительной") 
        #Теперь нам нужно обновить баланс, чтобы деньги поступили на счет
        self.__balance += amount
        #Запишем все в историю
        operation = {
            "type": "deposit",
            "amount": amount,
            "date": datetime.datetime.now(),
            "balance_after": self.__balance,
            "status": "success"
        }
        self.operations_history.append(operation)
    
    #Теперь добавляем метод withdraw
    def withdraw(self, amount):
        if amount <= 0:
            raise ValueError("Сумма снятия должна быть положительной")
        
        if amount > self.__balance:
            # наш клиент пытается снять больше, чем у него есть на счете
            operation = {
                "type": "withdraw",
                "amount": amount,
                "date": datetime.datetime.now(),
                "balance_after": self.__balance, 
                "status": "fail"
            }
            self.operations_history.append(operation)
            return False
        
        # если деньги есть, то их, конечно, можно снять
        self.__balance -= amount
        # теперь запишем все наши действия в историю
        operation = { 
            "type": "withdraw",
            "amount": amount,
            "date": datetime.datetime.now(),
            "balance_after": self.__balance,
            "status": "success"
        }
        self.operations_history.append(operation)
        return True
    
    # создаем метод get_history
    def get_history(self):
        return self.operations_history
    
    def get_balance(self):
        return self.__balance


# теперь создаем класс CreditAccount
class CreditAccount(Account):
    def __init__(self, account_holder, balance=0.0, credit_limit=0.0):
        # вызываем конструктор родительского класса
        super().__init__(account_holder, balance)
        # тут нам важно добавить открицательный баланс кредитки 
        self.__balance = float(balance)
        self.credit_limit = credit_limit
    
    def withdraw(self, amount):
        # Переопределяем метод с учётом кредитного лимита
        if amount <= 0:
            raise ValueError("Сумма снятия должна быть положительной")
        
        # максимум можно снять текущий баланс + кредитный лимит
        if amount > (self.__balance + self.credit_limit):
            operation = {
                "type": "withdraw",
                "amount": amount,
                "date": datetime.datetime.now(),
                "balance_after": self.__balance,
                "status": "fail",
                "used_credit": False  # кредит не использовался (операция не прошла)
            }
            self.operations_history.append(operation)
            return False
        
        used_credit = amount > self.__balance
        
        # Обновляем баланс
        self.__balance -= amount
        
        # Запись в историю
        operation = {
            "type": "withdraw",
            "amount": amount,
            "date": datetime.datetime.now(),
            "balance_after": self.__balance,
            "status": "success",
            "used_credit": used_credit  # True если использовали кредит
        }
        self.operations_history.append(operation)
        return True
    
    def available_credit(self):
        """Сколько ещё можно снять с учётом кредитного лимита"""
        return self.__balance + self.credit_limit


# теперь тестируем все случаи. Я учительница русского языка и литературы, поэтому буду проверять все на персонажах пьесы А. Островского "Бесприданница" (суммы вымышлены, но привязаны к характеристикам героев). Я обращусь к Харите Игнатьевне Огудаловой (той самой вдове, которая пыталась выгодно выдать замуж дочь Ларису), Ларисе Дмитриевне (дочери ее).
print("=" * 60)
print("ТЕСТИРОВАНИЕ: ИСТОРИЯ ОПЕРАЦИЙ")
print("=" * 60)

print("\n=== ТЕСТ Account ===")

# Создаём счёт и выполняем операции
print("\n1. Создание и операции со счётом Хариты Игнатьевны:")
acc = Account("Харита Игнатьевна", 300)
print(f"   Начальный баланс: {acc.get_balance()}")

acc.deposit(500)
print(f"   После deposit(500): {acc.get_balance()}")

acc.withdraw(300)
print(f"   После withdraw(300): {acc.get_balance()}")

acc.withdraw(5000)
print(f"   После неудачного withdraw(5000): {acc.get_balance()}")

# ПРОВЕРКА ИСТОРИИ ОПЕРАЦИЙ ДЛЯ Account
print("\n2. Проверка истории операций (Account):")
print("-" * 50)
history = acc.get_history()
print(f"Всего записей в истории: {len(history)}")

for i, operation in enumerate(history, 1):
    print(f"\nЗапись {i}:")
    print(f"  Тип операции: {operation['type']}")
    print(f"  Сумма: {operation['amount']}")
    print(f"  Дата и время: {operation['date']}")
    print(f"  Баланс после операции: {operation['balance_after']}")
    print(f"  Статус: {operation['status']}")

# Проверка структуры данных
print("\n3. Проверка структуры данных:")
if history:
    sample_op = history[0]
    print(f"  • Все ключи присутствуют: {sorted(sample_op.keys())}")
    print(f"  • Тип 'date' правильный: {type(sample_op['date']).__name__}")
    print(f"  • Тип 'status' правильный: {type(sample_op['status']).__name__}")

print("\n" + "=" * 60)
print("=== ТЕСТ CreditAccount ===")

# Создаём кредитный счёт
print("\n4. Создание кредитного счёта Ларисы Дмитриевны:")
credit_acc = CreditAccount("Лариса Дмитриевна", -500, 5000)
print(f"   Начальный баланс: {credit_acc.get_balance()}")
print(f"   Кредитный лимит: {credit_acc.credit_limit}")
print(f"   Доступный кредит: {credit_acc.available_credit()}")

# Операции с кредитным счётом
credit_acc.withdraw(800)
print(f"   После withdraw(800): {credit_acc.get_balance()}")

credit_acc.withdraw(10000)
print(f"   После неудачного withdraw(10000): {credit_acc.get_balance()}")

# ПРОВЕРКА ИСТОРИИ ОПЕРАЦИЙ ДЛЯ CreditAccount
print("\n5. Проверка истории операций (CreditAccount):")
print("-" * 50)
credit_history = credit_acc.get_history()
print(f"Всего записей в истории: {len(credit_history)}")

for i, operation in enumerate(credit_history, 1):
    print(f"\nЗапись {i}:")
    print(f"  Тип операции: {operation['type']}")
    print(f"  Сумма: {operation['amount']}")
    print(f"  Дата и время: {operation['date']}")
    print(f"  Баланс после операции: {operation['balance_after']}")
    print(f"  Статус: {operation['status']}")
    if 'used_credit' in operation:
        print(f"  Использован кредит: {operation['used_credit']}")

# Проверка поля used_credit
print("\n6. Проверка поля 'used_credit' в кредитном счёте:")
for i, operation in enumerate(credit_history, 1):
    if 'used_credit' in operation:
        print(f"  Запись {i}: used_credit = {operation['used_credit']} (тип: {type(operation['used_credit']).__name__})")
    else:
        print(f"  Запись {i}: поле 'used_credit' отсутствует")

print("\n" + "=" * 60)
print("ДОПОЛНИТЕЛЬНЫЙ ТЕСТ: ВСЕ ОПЕРАЦИИ СОХРАНЯЮТСЯ")
print("=" * 60)

# Создаём новый счёт для чистого теста
print("\n7. Детальная проверка сохранения истории:")
test_acc = Account("Тестовый Клиент", 100)
print(f"   Начальный баланс: {test_acc.get_balance()}")

# Фиксируем количество операций до
operations_before = len(test_acc.get_history())

# Выполняем несколько операций
operations_list = [
    ("deposit", 50),
    ("withdraw", 30),
    ("withdraw", 200),  # неудачная
    ("deposit", 100),
    ("withdraw", 80)
]

for op_type, amount in operations_list:
    if op_type == "deposit":
        test_acc.deposit(amount)
        print(f"   Выполнен deposit({amount}): баланс = {test_acc.get_balance()}")
    else:
        result = test_acc.withdraw(amount)
        status = "успешно" if result else "неудачно"
        print(f"   Выполнен withdraw({amount}): {status}, баланс = {test_acc.get_balance()}")

# Фиксируем количество операций после
operations_after = len(test_acc.get_history())
added_operations = operations_after - operations_before

print(f"\n8. Итоговая проверка:")
print(f"   Операций выполнено: {len(operations_list)}")
print(f"   Операций добавлено в историю: {added_operations}")
print(f"   Всего записей в истории: {operations_after}")

if added_operations == len(operations_list):
    print("   ВСЕ операции сохранены в истории!")
else:
    print("   НЕ ВСЕ операции сохранены в истории!")

# Выводим итоговую историю
print("\n9. Итоговая история операций:")
print("-" * 50)
for i, operation in enumerate(test_acc.get_history(), 1):
    print(f"{i:2}. {operation['type']:10} {operation['amount']:6} → {operation['status']:8} (баланс: {operation['balance_after']})")

print("\n" + "=" * 60)
print("ТЕСТИРОВАНИЕ ЗАВЕРШЕНО: ИСТОРИЯ ОПЕРАЦИЙ КОРРЕКТНО СОХРАНЯЕТСЯ!")
print("=" * 60)

ТЕСТИРОВАНИЕ: ИСТОРИЯ ОПЕРАЦИЙ

=== ТЕСТ Account ===

1. Создание и операции со счётом Хариты Игнатьевны:
   Начальный баланс: 300.0
   После deposit(500): 800.0
   После withdraw(300): 500.0
   После неудачного withdraw(5000): 500.0

2. Проверка истории операций (Account):
--------------------------------------------------
Всего записей в истории: 3

Запись 1:
  Тип операции: deposit
  Сумма: 500
  Дата и время: 2025-12-25 23:06:02.683000
  Баланс после операции: 800.0
  Статус: success

Запись 2:
  Тип операции: withdraw
  Сумма: 300
  Дата и время: 2025-12-25 23:06:02.684000
  Баланс после операции: 500.0
  Статус: success

Запись 3:
  Тип операции: withdraw
  Сумма: 5000
  Дата и время: 2025-12-25 23:06:02.684000
  Баланс после операции: 500.0
  Статус: fail

3. Проверка структуры данных:
  • Все ключи присутствуют: ['amount', 'balance_after', 'date', 'status', 'type']
  • Тип 'date' правильный: datetime
  • Тип 'status' правильный: str

=== ТЕСТ CreditAccount ===

4. Создание кред

<class 'ValueError'>: Начальный баланс не может быть отрицательным